In [1]:
import json
import os
import time

from multiprocessing import Pool

# local .json file with local path info
ungulates_root = os.path.dirname(os.getcwd())
local_paths_file = os.path.join(ungulates_root, 'local-paths.json')
with open(local_paths_file, "r") as json_file:
    local_paths = json.load(json_file)

import sys
sys.path.append(local_paths['overhead_functions_path'])

import koger_general_functions as kgf
import mapping_functions as kmap
from drone_movement import create_gt_segment_dicts, get_segment_drone_movement

In [2]:
observation_names = ['observation088']

In [3]:
all_segment_dicts = []

save_folder_name = 'worked_examples_drone_movement'

for observation_name in observation_names:
    print(f"processing {observation_name}")
    # This is where the raw video images are saved 
    frame_folders_root = os.path.join(local_paths['base_frames_folder'],
                                      observation_name)
    pix4d_folder = os.path.join(local_paths['pix4d_folder'], observation_name)
    output_folder = os.path.join(frame_folders_root, save_folder_name)
        
    pmatrix_file = os.path.join(pix4d_folder, "1_initial", "params",
                                f"worked_examples_{observation_name}_pmatrix.txt") 
    pmatrix_list = kmap.create_pmatrix_dicts(pmatrix_file)
        
    # Load the saved flight logs that contain both drone sensor info and the video frame info
    flight_logs_folder = os.path.join(pix4d_folder, 'drone-logs')
    flight_logs = kmap.get_cleaned_flight_logs(flight_logs_folder, pmatrix_list)
    
    segment_dicts = create_gt_segment_dicts(frame_folders_root, 
                                            flight_logs, output_folder, 
                                            save=True)
    all_segment_dicts.extend(segment_dicts)

processing observation088
2 gt images not used in final map.
Warning, /home/golden/kenya_drones/processing/kenya-tracking/raw-frames/raw-footage/observation088/worked_examples_drone_movement already exists
gt not found. using APR08_2018_A_DJI_0008_08224.jpg for APR08_2018_A_DJI_0008_08226.jpg.


In [ ]:
if __name__ == '__main__':
    t0 = time.time()
    with Pool(processes=8) as pool:
        pool.map(get_segment_drone_movement, all_segment_dicts)
    print('total time:', time.time() - t0)

Warning. Warp features below threshold: 14, seg num: 33.
